In [334]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Friday 10/07, 11:59 PM
## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

For Data 101 students, this project is worth 15% of your grade. For Info 258 students, this project is worth 12% of your grade.

- Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests.
- Public tests for multiple choice questions are for sanity check only (e.g. you are answering in the correct format). Partial credit will be awarded.
- Free-response questions will be manually graded. Please answer thoughtfully in complete sentences, drawing from knowledge in lectures and from your inspection of query plans.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.

Question | Points
--- | ---
0 | 1
1a	| 1
1bi	| 1
1bii	| 2
1c	| 1
1di	| 2
1dii	| 1
1ei	| 1
1eii	| 1
1eiii	| 1
1eiv	| 2
1ev	| 2
2a	| 1
2b	| 1
2c	| 2
2d	| 2
3a	| 1
3b	| 1
3c |	1
3d	| 3
4a	| 2
4b	| 1
4c	| 1
4di	| 2
4dii |	2
4ei	| 2
4eii | 	1
4eiii |	2
4eiv |	1
4ev |	1
4evi |	2
4evii |	2
5a | 2
5b | 2
5c | 1
5d | 2
6a | 2
6b | 1
6c | 1
6d | 2
6e | 2
7a | 1
7b | 1
7c | 2
8  | 6
**Total** | 72

In [335]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `ipython-sql` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [336]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

**Note:** If you run into the **role does not exist** error, feel free to ignore it. It does not affect data import.

In [337]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!gzip -dc data/lahman.pgdump.gz | psql -h localhost -d baseball -f -
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

ERROR:  database "baseball" is being accessed by other users
DETAIL:  There is 1 other session using the database.
ERROR:  database "baseball" already exists
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
CREATE EXTENSION
COMMENT
SET
psql:<stdin>:46: ERROR:  relation "allstarfull" already exists
psql:<stdin>:49: ERROR:  role "postgres" does not exist
psql:<stdin>:77: ERROR:  relation "appearances" already exists
psql:<stdin>:80: ERROR:  role "postgres" does not exist
psql:<stdin>:93: ERROR:  relation "awardsmanagers" already exists
psql:<stdin>:96: ERROR:  role "postgres" does not exist
psql:<stdin>:109: ERROR:  relation "awardsplayers" already exists
psql:<stdin>:112: ERROR:  role "postgres" does not exist
psql:<stdin>:126: ERROR:  relation "awardssharemanagers" already exists
psql:<stdin>:129: ERROR:  role "postgres" does not exist
psql:<stdin>:143: ERROR:  relation "awardsshareplayers" already exists
psql:<stdin>:146: ERROR:  role "postgres" does not exist
psql:<st

Name,Owner,Encoding,Collate,Ctype,Access privileges
baseball,jovyan,UTF8,en_US.utf8,en_US.utf8,None
imdb,jovyan,UTF8,en_US.utf8,en_US.utf8,None
jovyan,jovyan,UTF8,en_US.utf8,en_US.utf8,None
postgres,jovyan,UTF8,en_US.utf8,en_US.utf8,None
template0,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan
template1,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan


Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [338]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [339]:
%%sql
SELECT * FROM halloffame LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


Run the following cell for grading purposes.

In [340]:
!mkdir -p results

## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

As a reminder from Project 1, `%sql \d <table_name>` is helpful for identifying the columns in a table.

## Question 0: PostgreSQL Explain Analyze
**Please read through this section carefully, as a vast majority of the project will require you to inspect query plans via interpreting the output of the explain analyze command.**

To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_variable = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_variable"`

Take a look at the following sample query plan.

![title](data/sample_query.png)

It is highly recommended to read through [this article](https://www.cybertec-postgresql.com/en/how-to-interpret-postgresql-explain-analyze-output/) to see how you can interpret the output above. Everything before "Tools to interpret Explain Analyze output" is useful.

Here are some key things to note:
- There are two cost values: the first is the **startup cost** (cost to return the first row) and the second is the **total cost** (cost to return all rows).
- The unit for the estimated query cost is arbitrary (1 is the cost for reading an 8kB page during a sequential scan)
- When we ask you to identify the query cost, we are looking for the **total cost**
- When we ask you to identify the query time, we are looking for the **execution time** (in ms)
- Feel free to round the query cost / time to the nearest integer, but we'll accept anything more exact
- We recognzie that the execution time may vary between different cell executions, so the autograder will tolerate a reasonable range

Now, inspect the query plan above, and record the **cost** and **time** for the sample query.

<!--
BEGIN QUESTION
name: q0
points: 1
-->

In [341]:
sample_query = "SELECT * FROM people AS p INNER JOIN collegeplaying AS cp ON p.playerid = cp.playerid"
!psql -h localhost -d baseball -c "explain analyze $sample_query"

                                                         QUERY PLAN                                                          
-----------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=904.31..1274.57 rows=19346 width=167) (actual time=158.870..245.913 rows=17350 loops=1)
   Hash Cond: ((cp.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on collegeplaying cp  (cost=0.00..319.46 rows=19346 width=21) (actual time=0.016..71.724 rows=17350 loops=1)
   ->  Hash  (cost=650.25..650.25 rows=20325 width=146) (actual time=158.600..158.604 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 3633kB
         ->  Seq Scan on people p  (cost=0.00..650.25 rows=20325 width=146) (actual time=0.017..147.432 rows=19370 loops=1)
 Planning Time: 1.793 ms
 Execution Time: 247.039 ms
(8 rows)



In [342]:
sample_query_cost = 1193
sample_query_timing = 185

In [343]:
grader.check("q0")

q0 results: All test cases passed!

## Question 1: Queries and Views
### Question 1a:
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **NOT** use any sub-queries.

In [344]:
%%sql result_1a <<
SELECT namefirst, namelast, h.playerid, h.yearid
FROM halloffame as h INNER JOIN people as p on h.playerid = p.playerid
WHERE inducted = 'Y'

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable result_1a


In [345]:
# Do not delete/edit this cell
result_1a.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_1a.csv', index=False)

In [346]:
grader.check("q1a")

q1a results: All test cases passed!

### Question 1b:
In this question, we will compare the query you wrote in `Question 1a` against the provided query in `Question 1bi` by inspecting the query plans.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a`. Record the **execution time** and **cost** for each query.

In [347]:
%%sql provided_query <<
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable provided_query


In [348]:
provided_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof WHERE hof.inducted = 'Y' AND p.playerid = hof.playerid;"
!psql -h localhost -d baseball -c "explain analyze $provided_query_str"

                                                       QUERY PLAN                                                        
-------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=904.31..1022.57 rows=390 width=25) (actual time=9.721..10.386 rows=323 loops=1)
   Hash Cond: ((halloffame.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on halloffame  (cost=0.00..117.24 rows=390 width=13) (actual time=0.015..0.579 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Hash  (cost=650.25..650.25 rows=20325 width=21) (actual time=9.493..9.494 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 1293kB
         ->  Seq Scan on people p  (cost=0.00..650.25 rows=20325 width=21) (actual time=0.009..4.921 rows=19370 loops=1)
 Planning Time: 0.988 ms
 Execution Time: 10.500 ms
(10 rows)



In [349]:
inducted_hof_query_str = "SELECT namefirst, namelast, h.playerid, h.yearid FROM halloffame as h INNER JOIN people as p on h.playerid = p.playerid WHERE inducted = 'Y'"
!psql -h localhost -d baseball -c "explain analyze $inducted_hof_query_str"

                                                       QUERY PLAN                                                        
-------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=904.31..1022.57 rows=390 width=25) (actual time=9.611..10.329 rows=323 loops=1)
   Hash Cond: ((h.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on halloffame h  (cost=0.00..117.24 rows=390 width=13) (actual time=0.014..0.624 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Hash  (cost=650.25..650.25 rows=20325 width=21) (actual time=9.433..9.434 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 1293kB
         ->  Seq Scan on people p  (cost=0.00..650.25 rows=20325 width=21) (actual time=0.010..4.971 rows=19370 loops=1)
 Planning Time: 0.957 ms
 Execution Time: 10.432 ms
(10 rows)



In [350]:
provided_query_cost = 959.06
provided_query_timing = 24.795
your_query_cost = 959.06
your_query_timing = 13.625

In [351]:
grader.check("q1bi")

q1bi results: All test cases passed!

#### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of answer choices which are true statements.

1. Consider the following statements:
    1. Both the queries have the same cost.
    1. The provided query has a faster execution time because it makes use of a subquery.
    1. The query you wrote has a faster execution time because it does not make use a subquery.
    1. The provided query has less cost because it makes use of a subquery.
    1. The query you wrote has less cost because it does not make use a subquery.
    1. The queries have the same output.
    1. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1bii
points: 2
-->

In [352]:
q1b_part2 = ['A','C','F']

In [353]:
grader.check("q1bii")

q1bii results: All test cases passed!

### Question 1c:
Write a query that creates a view named `inducted_hof_ca` of the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

In [354]:
%%sql result_1c <<
DROP VIEW IF EXISTS inducted_hof_ca CASCADE;
CREATE VIEW inducted_hof_ca AS
SELECT p.namefirst, p.namelast,p.playerid ,j.schoolid, j.yearid
FROM People AS p INNER JOIN 
(SELECT h.playerid, c.schoolid, h.yearid
FROM halloffame AS h INNER JOIN 
 (SELECT c.playerid, c.schoolid, c.yearid
FROM schools AS s INNER JOIN collegeplaying AS c on s.schoolid = c.schoolid
WHERE schoolstate = 'CA') AS c on h.playerid = c.playerid
WHERE inducted = 'Y') AS j ON p.playerid = j.playerid 
ORDER BY yearid,playerid;

SELECT * FROM inducted_hof_ca;


 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
23 rows affected.
Returning data to local variable result_1c


In [355]:
# Do not delete/edit this cell
result_1c.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1c.csv', index=False)

In [356]:
grader.check("q1c")

q1c results: All test cases passed!

### Question 1d:
For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

#### Question 1di:
Write two queries that accomplish this task -- one query using the view you created in the `Question 1c` and one query that does not use the view, common table expressions (CTEs), or any sub-queries.

In [357]:
%%sql with_view <<

SELECT schoolid, yearid, count(*)
FROM inducted_hof_ca
GROUP BY schoolid, yearid
ORDER BY yearid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_view


In [358]:
%%sql without_view <<
SELECT a.schoolid, a.yearid, count(*)
FROM (SELECT p.namefirst, p.namelast,p.playerid ,j.schoolid, j.yearid
FROM People AS p INNER JOIN 
(SELECT h.playerid, c.schoolid, h.yearid
FROM halloffame AS h INNER JOIN 
 (SELECT c.playerid, c.schoolid, c.yearid
FROM schools AS s INNER JOIN collegeplaying AS c on s.schoolid = c.schoolid
WHERE schoolstate = 'CA') AS c on h.playerid = c.playerid
WHERE inducted = 'Y') AS j ON p.playerid = j.playerid 
ORDER BY yearid,playerid) AS a
GROUP BY schoolid, yearid
ORDER BY yearid

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable without_view


In [359]:
# Do not delete/edit this cell
with_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_view.csv', index=False)
without_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_no_view.csv', index=False)

In [360]:
grader.check("q1di")

q1di results: All test cases passed!

#### Question 1dii:
Fill in your queries from `Question 1di` and inspect the query plans for the two queries. Record the execution time and cost for each query.

In [361]:
with_view_query_str = 'SELECT schoolid, yearid, count(*) FROM inducted_hof_ca GROUP BY schoolid, yearid ORDER BY yearid'
!psql -h localhost -d baseball -c "explain analyze $with_view_query_str"

                                                                             QUERY PLAN                                                                              
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=613.88..621.50 rows=116 width=20) (actual time=5.699..5.711 rows=13 loops=1)
   Group Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
   ->  Incremental Sort  (cost=613.88..619.47 rows=116 width=12) (actual time=5.693..5.697 rows=23 loops=1)
         Sort Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
         Presorted Key: inducted_hof_ca.yearid
         Full-sort Groups: 1  Sort Method: quicksort  Average Memory: 26kB  Peak Memory: 26kB
         ->  Subquery Scan on inducted_hof_ca  (cost=613.85..615.30 rows=116 width=12) (actual time=5.651..5.658 rows=23 loops=1)
               ->  Sort  (cost=613.85..614.14 rows=116 wid

In [362]:
without_view_query_str = "SELECT a.schoolid, a.yearid, count(*) FROM (SELECT p.namefirst, p.namelast,p.playerid ,j.schoolid, j.yearid FROM People AS p INNER JOIN  (SELECT h.playerid, c.schoolid, h.yearid FROM halloffame AS h INNER JOIN  (SELECT c.playerid, c.schoolid, c.yearid FROM schools AS s INNER JOIN collegeplaying AS c on s.schoolid = c.schoolid WHERE schoolstate = 'CA') AS c on h.playerid = c.playerid WHERE inducted = 'Y') AS j ON p.playerid = j.playerid  ORDER BY yearid,playerid) AS a GROUP BY schoolid, yearid ORDER BY yearid"
!psql -h localhost -d baseball -c"explain analyze $without_view_query_str"

                                                                             QUERY PLAN                                                                              
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=613.88..621.50 rows=116 width=20) (actual time=5.599..5.611 rows=13 loops=1)
   Group Key: a.yearid, a.schoolid
   ->  Incremental Sort  (cost=613.88..619.47 rows=116 width=12) (actual time=5.594..5.598 rows=23 loops=1)
         Sort Key: a.yearid, a.schoolid
         Presorted Key: a.yearid
         Full-sort Groups: 1  Sort Method: quicksort  Average Memory: 26kB  Peak Memory: 26kB
         ->  Subquery Scan on a  (cost=613.85..615.30 rows=116 width=12) (actual time=5.541..5.548 rows=23 loops=1)
               ->  Sort  (cost=613.85..614.14 rows=116 width=257) (actual time=5.540..5.544 rows=23 loops=1)
                     Sort Key: h.

In [363]:
with_view_cost = 547.39
with_view_timing = 6.461 
without_view_cost = 547.39
without_view_timing = 9.334

In [364]:
grader.check("q1dii")

q1dii results: All test cases passed!

### Question 1e:
#### Question 1ei:
Now, let's try creating a materialized view named `inducted_hof_ca_mat` instead of the regular view from `Question 1c`.

In [365]:
%%sql inducted_hof_ca_materialized <<
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS
SELECT p.namefirst, p.namelast,p.playerid ,j.schoolid, j.yearid
FROM People AS p INNER JOIN 
(SELECT h.playerid, c.schoolid, h.yearid
FROM halloffame AS h INNER JOIN 
 (SELECT c.playerid, c.schoolid, c.yearid
FROM schools AS s INNER JOIN collegeplaying AS c on s.schoolid = c.schoolid
WHERE schoolstate = 'CA') AS c on h.playerid = c.playerid
WHERE inducted = 'Y') AS j ON p.playerid = j.playerid 
ORDER BY yearid,playerid;

SELECT * FROM inducted_hof_ca_mat;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
23 rows affected.
23 rows affected.
Returning data to local variable inducted_hof_ca_materialized


In [366]:
# Do not delete/edit this cell
inducted_hof_ca_materialized.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1ei.csv', index=False)

In [367]:
grader.check("q1ei")

q1ei results: All test cases passed!

#### Question 1eii:

Now, rewrite the query from `Question 1d` to use the materialized view `inducted_hof_ca_mat`.

In [368]:
%%sql with_materialized_view <<
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS
SELECT schoolid, yearid, count(*)
FROM inducted_hof_ca
GROUP BY schoolid, yearid
ORDER BY yearid;

SELECT * FROM inducted_hof_ca_mat;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
13 rows affected.
13 rows affected.
Returning data to local variable with_materialized_view


In [369]:
# Do not delete/edit this cell
with_materialized_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1eii.csv', index=False)

In [370]:
grader.check("q1eii")

q1eii results: All test cases passed!

#### Question 1eiii:
Inspect the query plan and record the execution time and cost of the query that uses the materialized view.

In [371]:
with_materialized_view_query_str = "SELECT schoolid, yearid, count(*) FROM inducted_hof_ca GROUP BY schoolid, yearid ORDER BY yearid;"
!psql -h localhost -d baseball -c "explain analyze $with_materialized_view_query_str"

                                                                             QUERY PLAN                                                                              
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=613.88..621.50 rows=116 width=20) (actual time=5.295..5.304 rows=13 loops=1)
   Group Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
   ->  Incremental Sort  (cost=613.88..619.47 rows=116 width=12) (actual time=5.290..5.294 rows=23 loops=1)
         Sort Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
         Presorted Key: inducted_hof_ca.yearid
         Full-sort Groups: 1  Sort Method: quicksort  Average Memory: 26kB  Peak Memory: 26kB
         ->  Subquery Scan on inducted_hof_ca  (cost=613.85..615.30 rows=116 width=12) (actual time=5.254..5.260 rows=23 loops=1)
               ->  Sort  (cost=613.85..614.14 rows=116 wid

In [372]:
with_materialized_view_cost = 547.39
with_materialized_view_timing =  6.001

In [373]:
grader.check("q1eiii")

q1eiii results: All test cases passed!

#### Question 1eiv:
Given your findings from inspecting the query plans, as well as your understanding of views and materialized views from lectures, answer the following question. Assign the variable `q1e_part4` to a list of all statements which are true.

1. Consider the following statements:
    1. Views will reduce the execution time and the cost of a query.
    1. Views will reduce the execution time of a query, but not the cost.
    1. Views will reduce the cost of a query, but not the execution time.
    1. Materialized views reduce the execution time and the cost of a query.
    1. Materialized views reduce the execution time, but not cost of a query.
    1. Materialized views reduce the cost of a query, but not the execution time.
    1. Materialized views will result in the same query plan as a query using views.
    1. Materialized views and views take the same time to create.
    1. Materialized views take less time to create than a view.
    1. Materialized views take more time to create than a view.
    
*Note:* Your answer should look like `q1e_part4 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1eiv
points: 2
-->

In [374]:
with_view_cost = 547.39
with_view_timing = 6.461 
without_view_cost = 547.39
without_view_timing = 9.334
with_materialized_view_cost = 547.39
with_materialized_view_timing =  6.001

In [375]:
q1e_part4 = ['E','I']

In [376]:
grader.check("q1eiv")

q1eiv results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 1ev:

Explain your answer to the previous part based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q1ev
manual: true
points: 2
-->

Because regular view does not store output on the disk and runs the same query so it has same query cost and same performance. On the other hand, materialized view stores output on the disk so it takes more memory which leads to having high query cost but it is faster at executing queries because the underlying query does not need to be rerun each time the view is called so it has low execution time.

<!-- END QUESTION -->



## Question 2: Predicate Pushdown
In this question, we will explore the impact of predicates on a query's execution, particularly inspecting when the optimizer applies predicates.

### Question 2a:
Recall the `inducted_hof_ca` created in `Question 1c`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost.

In [377]:
query_view_str = "SELECT p.namefirst, p.namelast,p.playerid ,j.schoolid, j.yearid FROM People AS p INNER JOIN (SELECT h.playerid, c.schoolid, h.yearid FROM halloffame AS h INNER JOIN  (SELECT c.playerid, c.schoolid, c.yearid FROM schools AS s INNER JOIN collegeplaying AS c on s.schoolid = c.schoolid WHERE schoolstate = 'CA') AS c on h.playerid = c.playerid WHERE inducted = 'Y') AS j ON p.playerid = j.playerid ORDER BY yearid,playerid;"
!psql -h localhost -d baseball -c "explain analyze $query_view_str"

                                                                    QUERY PLAN                                                                     
---------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=624.04..624.33 rows=116 width=33) (actual time=6.144..6.148 rows=23 loops=1)
   Sort Key: h.yearid, p.playerid
   Sort Method: quicksort  Memory: 26kB
   ->  Nested Loop  (cost=450.63..620.06 rows=116 width=33) (actual time=5.596..6.104 rows=23 loops=1)
         Join Filter: ((h.playerid)::text = (p.playerid)::text)
         ->  Hash Join  (cost=450.34..570.69 rows=116 width=30) (actual time=5.556..5.907 rows=23 loops=1)
               Hash Cond: ((h.playerid)::text = (c.playerid)::text)
               ->  Seq Scan on halloffame h  (cost=0.00..117.24 rows=390 width=13) (actual time=0.015..0.645 rows=323 loops=1)
                     Filter: ((inducted)::text = 'Y'::text)
           

In [378]:
query_view_cost = 530.78
query_view_timing = 6.411

In [379]:
grader.check("q2a")

q2a results: All test cases passed!

### Question 2b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [380]:
%%sql result_2b <<
SELECT *
FROM inducted_hof_ca
WHERE yearid > 2010;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8 rows affected.
Returning data to local variable result_2b


In [381]:
query_view_with_filter_str = "SELECT * FROM inducted_hof_ca WHERE yearid > 2010;"
!psql -h localhost -d baseball -c "explain analyze $query_view_with_filter_str"

                                                                             QUERY PLAN                                                                              
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=274.84..274.86 rows=8 width=33) (actual time=1.200..1.202 rows=8 loops=1)
   Sort Key: h.yearid, p.playerid
   Sort Method: quicksort  Memory: 25kB
   ->  Nested Loop  (cost=0.85..274.72 rows=8 width=33) (actual time=0.710..1.158 rows=8 loops=1)
         Join Filter: ((h.playerid)::text = (p.playerid)::text)
         ->  Nested Loop  (cost=0.57..271.31 rows=8 width=30) (actual time=0.683..1.078 rows=8 loops=1)
               ->  Nested Loop  (cost=0.29..250.50 rows=68 width=30) (actual time=0.666..0.963 rows=27 loops=1)
                     ->  Seq Scan on halloffame h  (cost=0.00..129.88 rows=26 width=13) (actual time=0.632..0.715 rows=31 loops=1

In [382]:
query_view_with_filter_cost = 209.93
query_view_with_filter_timing = 1.223

In [383]:
grader.check("q2b")

q2b results: All test cases passed!

### Question 2c:
Given your findings from inspecting the query plans of queries from `Question 2a-b`, answer the following question. Assign the variable `q2c` to a list of all statements which are true.

1. Consider the following statements:
    1. Adding a filter lowered the cost.
    1. Adding a filter increased the cost.
    1. Adding a filter did not change the cost.
    1. Adding a filter increased the execution time.
    1. Adding a filter decreased the execution time.
    1. Adding a filter did not change the execution time.
    1. No statement is true.
    
    
**Note:** Your answer should look like `q2c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [384]:
q2c = ['A','E']

In [385]:
grader.check("q2c")

q2c results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 2d:

Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q2d
manual: true
points: 2
-->

Filtering pushes down the WHERE predicate into the view before the join, which reduces the table size for the view to join, thereby saving query cost. 

<!-- END QUESTION -->



## Question 3: Join Approaches

In this question, we'll explore different join approaches (Nested Loop Join, Merge Join, Hash Join) and discuss how the query optimizer picks the best approach.

### Question 3a:
Perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In [386]:
%%sql q3a_join <<
SELECT *
FROM people p INNER JOIN collegeplaying c ON p.playerid = c.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3a_join


Inspect the query plan for the query you wrote above.

In [387]:
q3a_query_str = "SELECT * FROM people p INNER JOIN collegeplaying c ON p.playerid = c.playerid;"
!psql -h localhost -d baseball -c "explain analyze $q3a_query_str"

                                                        QUERY PLAN                                                         
---------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=904.31..1274.57 rows=19346 width=167) (actual time=11.313..26.036 rows=17350 loops=1)
   Hash Cond: ((c.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on collegeplaying c  (cost=0.00..319.46 rows=19346 width=21) (actual time=0.012..2.037 rows=17350 loops=1)
   ->  Hash  (cost=650.25..650.25 rows=20325 width=146) (actual time=11.109..11.112 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 3633kB
         ->  Seq Scan on people p  (cost=0.00..650.25 rows=20325 width=146) (actual time=0.010..2.866 rows=19370 loops=1)
 Planning Time: 1.227 ms
 Execution Time: 26.969 ms
(8 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3a` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3a = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3a
points: 1
-->

In [388]:
q3a = ['C']

In [389]:
grader.check("q3a")

q3a results: All test cases passed!

### Question 3b

Similar to Question 3a, perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In addition, **sort your output by `playerid`.**

In [390]:
%%sql q3b_join <<
SELECT *
FROM people p INNER JOIN collegeplaying c ON p.playerid = c.playerid
ORDER BY p.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable q3b_join


Inspect the query plan for the query you wrote above.

In [391]:
q3b_query_str = "SELECT * FROM people p INNER JOIN collegeplaying c ON p.playerid = c.playerid ORDER BY p.playerid;"
!psql -h localhost -d baseball -c "explain analyze $q3b_query_str"

                                                                         QUERY PLAN                                                                         
------------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=0.57..2034.58 rows=19346 width=167) (actual time=0.050..17.605 rows=17350 loops=1)
   Merge Cond: ((p.playerid)::text = (c.playerid)::text)
   ->  Index Scan using master_pkey on people p  (cost=0.29..1059.71 rows=20325 width=146) (actual time=0.012..3.965 rows=19368 loops=1)
   ->  Index Only Scan using collegeplaying_pkey on collegeplaying c  (cost=0.29..682.23 rows=19346 width=21) (actual time=0.033..2.650 rows=17350 loops=1)
         Heap Fetches: 117
 Planning Time: 0.932 ms
 Execution Time: 18.234 ms
(7 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3b` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3b = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [392]:
q3b = ['B']

In [393]:
grader.check("q3b")

q3b results: All test cases passed!

### Question 3c
Write a query to retrieve all possible player pair combinations. Project all columns, but **limit to 1000 rows** to ensure your query doesn't take an exorbitant amount of time to run.

**Hint:** You can do this by performing an inner join of the `people` table on itself with an inequality condition.

In [394]:
%%sql q3c_join <<
SELECT *
FROM people p1 INNER JOIN people p2 ON p1.playerid != p2.playerid
LIMIT 1000;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1000 rows affected.
Returning data to local variable q3c_join


Inspect the query plan for the query you wrote above.

In [395]:
q3c_query_str = "SELECT * FROM people p1 INNER JOIN people p2 ON p1.playerid != p2.playerid LIMIT 1000;"
!psql -h localhost -d baseball -c "explain analyze $q3c_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Limit  (cost=0.00..15.00 rows=1000 width=292) (actual time=0.045..0.942 rows=1000 loops=1)
   ->  Nested Loop  (cost=0.00..6197935.69 rows=413085300 width=292) (actual time=0.044..0.880 rows=1000 loops=1)
         Join Filter: ((p1.playerid)::text <> (p2.playerid)::text)
         Rows Removed by Join Filter: 1
         ->  Seq Scan on people p1  (cost=0.00..650.25 rows=20325 width=146) (actual time=0.027..0.027 rows=1 loops=1)
         ->  Materialize  (cost=0.00..751.88 rows=20325 width=146) (actual time=0.009..0.459 rows=1001 loops=1)
               ->  Seq Scan on people p2  (cost=0.00..650.25 rows=20325 width=146) (actual time=0.001..0.122 rows=1001 loops=1)
 Planning Time: 0.842 ms
 Execution Time: 1.105 ms
(9 rows)



Given your findings from inspecting the query plan, answer the following question. Assign the variable `q3c` to a list of all the options which are true for the proposed statement.

1. Which join approach did the query optimizer choose?
    1. Nested Loop Join
    1. Merge Join
    1. Hash Join
    1. None of the Above

**Note:** Your answer should be formatted as follows: `q3c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [396]:
q3c = ['A']

In [397]:
grader.check("q3c")

q3c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 3d
Please discuss your findings in the previous parts. In particular, we are interested in hearing why you think the query optimizer chooses the ultimate join approach in each of the above three scenarios. Feel free to discuss the pros and cons of each join approach as well.

If you feel stuck, here are some things to consider: Does a non-equijoin constrain us to certain join approaches? What's an added benefit in regards to the output of merge join?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 3
-->

In 3a, the query optimizer chose Hash join because it is the fastest join among other joins when input tables are quite large and no adequate indexes exist on them.
In 3b, the optimizer chose to do Merge Join because the query required to be sorted by playerid and Merge Join requires the input data to be sorted, therefore, it chose Merge Join. 
In 3c, the optimizer chose to do Nested Loop join because it needs to go through every row to get the possible combinations of player using non-equijoin where other joins can not compare inequality condition. 

<!-- END QUESTION -->



## Question 4: Indexes

### Question 4a:
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [398]:
%%sql result_4a <<
SELECT playerid, AVG(salary)
FROM appearances NATURAL JOIN salaries 
WHERE g_batting = 10
GROUP BY playerid


 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
134 rows affected.
Returning data to local variable result_4a


In [399]:
# Do not delete/edit this cell
result_4a.DataFrame().sort_values('playerid').to_csv('results/result_4a.csv', index=False)

In [400]:
grader.check("q4ai")

q4ai results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [401]:
result_4a_query_str = "SELECT playerid, AVG(salary) FROM appearances NATURAL JOIN salaries WHERE g_batting = 10 GROUP BY playerid"
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                      
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=4731.98..4732.00 rows=1 width=17) (actual time=19.895..19.978 rows=134 loops=1)
   Group Key: appearances.playerid
   ->  Sort  (cost=4731.98..4731.98 rows=1 width=17) (actual time=19.884..19.895 rows=138 loops=1)
         Sort Key: appearances.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Merge Join  (cost=1723.11..4731.97 rows=1 width=17) (actual time=5.080..19.672 rows=138 loops=1)
               Merge Cond: ((salaries.yearid = appearances.yearid) AND ((sa

In [402]:
result_4a_cost = 3636.97
result_4a_timing = 22.475

In [403]:
grader.check("q4aii")

q4aii results: All test cases passed!

### Question 4b:
Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [404]:
%%sql result_4b <<
CREATE INDEX g_batting_idx
ON appearances(g_batting)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4b


Now, re-inspect the query plan of the query from `Question 4a` and record its execution time and cost.

In [405]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                      
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=4727.61..4727.64 rows=1 width=17) (actual time=20.648..20.763 rows=134 loops=1)
   Group Key: appearances.playerid
   ->  Sort  (cost=4727.61..4727.62 rows=1 width=17) (actual time=20.637..20.651 rows=138 loops=1)
         Sort Key: appearances.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Merge Join  (cost=1718.81..4727.60 rows=1 width=17) (actual time=5.304..20.427 rows=138 loops=1)
               Merge Cond: ((salaries.yearid = appearances.yearid) AND ((sa

In [406]:
result_4b_cost = 2379.37
result_4b_timing =12.023 

In [407]:
grader.check("q4b")

q4b results: All test cases passed!

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, **drop the index before moving onto the next question.**

In [408]:
%%sql 
DROP INDEX IF EXISTS g_batting_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4c:
Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table.

In [409]:
%%sql result_4c <<
CREATE INDEX salary_idx
ON salaries(salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
(psycopg2.errors.DuplicateTable) relation "salary_idx" already exists

[SQL: CREATE INDEX salary_idx
ON salaries(salary)]
(Background on this error at: https://sqlalche.me/e/14/f405)


Re-inspect the query plan of the query from the `Question 4a` and record its execution time and cost.

In [410]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                                                                     QUERY PLAN                                                                                                                      
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=4727.61..4727.64 rows=1 width=17) (actual time=12.760..12.832 rows=134 loops=1)
   Group Key: appearances.playerid
   ->  Sort  (cost=4727.61..4727.62 rows=1 width=17) (actual time=12.752..12.760 rows=138 loops=1)
         Sort Key: appearances.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Merge Join  (cost=1718.81..4727.60 rows=1 width=17) (actual time=3.915..12.630 rows=138 loops=1)
               Merge Cond: ((salaries.yearid = appearances.yearid) AND ((sa

In [411]:
result_4c_cost = 3636.97
result_4c_timing = 22.284

In [412]:
grader.check("q4c")

q4c results: All test cases passed!

### Question 4d:
#### Question 4di:
Given your findings from inspecting the query plans with no indexes, an index on `g_batting`, and an index on `salary`, answer the following question. Assign the variable `q4d_part1` to a list of all options which are true for the proposed statement.

1. Consider the following statements:
    1. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.
    1. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `g_batting` index did have a significant impact on the query cost and execution time.
    1. Adding the `salary` index did not have a significant impact on the query execution time and cost.
    1. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as follows: `q4d_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4di
points: 2
-->

In [413]:
q4d_part1 = ['D','E ']

In [414]:
grader.check("q4di")

q4di results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4dii:

Explain your answer based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4dii
manual: true
points: 2
-->

I chose D, because adding the g_batting index improves the performance of the query in finding specific row quickly since the query asks to find WHERE g_batting = 10. This can be found quickly because g_batting index sorts g_batting in ascending order so it would decrease the amount of rows to scan compared to not having an index on g_batting. I also chose E, becuase adding the salary index is not beneficial when the query uses g_batting and playerid to do searching. Therefore, it does not improve the cost nor time.

<!-- END QUESTION -->



In the following question, we will further explore the impact of indexes on query performance. To avoid any interference by the `salary_idx` index, please drop the index before moving onto the next question.

In [415]:
%%sql 
DROP INDEX IF EXISTS salary_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4e:
#### Question 4ei:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games and batted in 10 games (the number of games in which a player played can be found in the `g_all` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [416]:
%%sql result_4ei <<
SELECT playerid,yearid,salary
FROM appearances NATURAL JOIN salaries
WHERE g_batting = 10 AND g_all = 10

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
120 rows affected.
Returning data to local variable result_4ei


In [417]:
# Do not delete/edit this cell
result_4ei.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4ei.csv', index=False)

In [418]:
grader.check("q4ei_part1")

q4ei_part1 results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [419]:
result_4ei_query_str = "SELECT playerid,yearid,salary FROM appearances NATURAL JOIN salaries WHERE g_batting = 10 AND g_all = 10"
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                                                QUERY PLAN                                                                                                 
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=4.93..259.00 rows=1 width=21) (actual time=4.701..8.592 rows=120 loops=1)
   ->  Bitmap Heap Scan on appearances  (cost=4.51..81.60 rows=21 width=20) (actual time=0.392..2.933 rows=1289 loops=1)
         Recheck Cond: ((g_batting = 10) AND (g_all = 10))
         Heap Blocks: exact=869
         ->  Bitmap Index Scan on g_batting_g_all_idx  (cost=0.00..4.50 rows=21 width=0) (actual time=0.226..0.226 rows=1289 loops=1)
               Index Cond: ((g_batting = 10) AND (g_all = 10))
   ->  Index Scan using salaries_pkey on salaries  (cost=0.42..8.45 rows=1 width=27) (act

In [420]:
result_4ei_cost =240.34
result_4ei_timing = 12.810

In [421]:
grader.check("q4ei_part2")

q4ei_part2 results: All test cases passed!

#### Question 4eii:
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the query from `Question 4ei` and record the execution time and cost.

In [422]:
%%sql result_4eii << 
CREATE INDEX g_batting_idx
ON appearances(g_batting)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eii


In [423]:
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                                                                QUERY PLAN                                                                                                 
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=4.93..259.00 rows=1 width=21) (actual time=4.627..8.379 rows=120 loops=1)
   ->  Bitmap Heap Scan on appearances  (cost=4.51..81.60 rows=21 width=20) (actual time=0.383..2.763 rows=1289 loops=1)
         Recheck Cond: ((g_batting = 10) AND (g_all = 10))
         Heap Blocks: exact=869
         ->  Bitmap Index Scan on g_batting_g_all_idx  (cost=0.00..4.50 rows=21 width=0) (actual time=0.216..0.217 rows=1289 loops=1)
               Index Cond: ((g_batting = 10) AND (g_all = 10))
   ->  Index Scan using salaries_pkey on salaries  (cost=0.42..8.45 rows=1 width=27) (act

In [424]:
result_4eii_with_index_cost = 240.34
result_4eii_with_index_timing = 6.782

In [425]:
grader.check("q4eii")

q4eii results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [426]:
%%sql 
DROP INDEX IF EXISTS g_batting_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4eiii:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [427]:
%%sql result_4eiii <<
SELECT playerid,yearid,salary
FROM appearances NATURAL JOIN salaries
WHERE g_batting = 10 OR g_all = 10

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
216 rows affected.
Returning data to local variable result_4eiii


In [428]:
# Do not delete/edit this cell
result_4eiii.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4eiii.csv', index=False)

In [429]:
grader.check("q4eiii_part1")

q4eiii_part1 results: All test cases passed!

In [430]:
result_4eiii_query_str = "SELECT playerid,yearid,salary FROM appearances NATURAL JOIN salaries WHERE g_batting = 10 OR g_all = 10"
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                                
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=3044.99..6061.87 rows=1 width=21) (actual time=7.608..23.616 rows=216 loops=1)
   Merge Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Index Scan using salaries_pkey on salaries  (cost=0.42..35255.03 rows=326428 width=27) (actual time=0.048..10.045 rows=26429 loops=1)
   ->  Sort  (cost=3022.73..3030.08 rows=2940 width=20) (act

In [431]:
result_4eiii_cost = 3667.81
result_4eiii_timing = 14.429

In [432]:
grader.check("q4eiii_part2")

q4eiii_part2 results: All test cases passed!

#### Question 4eiv
Now, let's see the impact of adding an index on `g_batting` column will have on the query. Re-create the index and re-inspect the query from `Question 4eiii` and record the execution time and cost.

In [433]:
%%sql result_4eiv << 
CREATE INDEX g_batting_idx 
ON appearances(g_batting)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eiv


In [434]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                                
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=3044.99..6061.87 rows=1 width=21) (actual time=5.485..15.685 rows=216 loops=1)
   Merge Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Index Scan using salaries_pkey on salaries  (cost=0.42..35255.03 rows=326428 width=27) (actual time=0.048..6.586 rows=26429 loops=1)
   ->  Sort  (cost=3022.73..3030.08 rows=2940 width=20) (actu

In [435]:
result_4eiv_with_index_cost = 3667.81
result_4eiv_with_index_timing = 13.631

In [436]:
grader.check("q4eiv")

q4eiv results: All test cases passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [437]:
%%sql 
DROP INDEX IF EXISTS g_batting_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4ev:
Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost.

In [438]:
%%sql result_4v <<
CREATE INDEX g_batting_g_all_idx
ON appearances(g_batting, g_all)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
(psycopg2.errors.DuplicateTable) relation "g_batting_g_all_idx" already exists

[SQL: CREATE INDEX g_batting_g_all_idx
ON appearances(g_batting, g_all)]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [439]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                                                               QUERY PLAN                                                                                                                
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Merge Join  (cost=3044.99..6061.87 rows=1 width=21) (actual time=8.216..24.502 rows=216 loops=1)
   Merge Cond: ((salaries.yearid = appearances.yearid) AND ((salaries.teamid)::text = (appearances.teamid)::text) AND ((salaries.lgid)::text = (appearances.lgid)::text) AND ((salaries.playerid)::text = (appearances.playerid)::text))
   ->  Index Scan using salaries_pkey on salaries  (cost=0.42..35255.03 rows=326428 width=27) (actual time=0.071..10.395 rows=26429 loops=1)
   ->  Sort  (cost=3022.73..3030.08 rows=2940 width=20) (act

In [440]:
result_4ev_multiple_col_index_cost = 3667.81
result_4ev_multiple_col_index_timing = 13.666

In [441]:
grader.check("q4ev")

q4ev results: All test cases passed!

#### Question 4evi:
Given your findings from inspecting the query plans from `Question 4e`, answer the following question. Assign the variable `q4e_part6` to a list of all statements that are true.

1. Consider the following statements:
    1. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as follows: `q4e_part6 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4evi
points: 2
-->

In [442]:
q4e_part6 = ['C','I']

In [443]:
grader.check("q4evi")

q4evi results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 4evii

Explain your answer to the previous part based on your knowledge from lectures, and details of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4evii
manual: true
points: 2
-->

I chose the answer C, because adding an index on a column used in an AND predicate reduced the query cost (3928.31 to 3296.09) and the execution time (24.273 to 18.581). This is reasonable because there will be less rows to be added on the database compared to OR. Also, I chose 'I' as the answer becuase the execution time and cost was lower than compared to 4eiv. This is reasonable because Multicolumn indexes can store additional sorted pointers to other columns and can retrieve data faster from the table.

<!-- END QUESTION -->



## Question 5: More Indexes
### Question 5a:
Write 2 queries: one that finds the minimum salary from the salary table and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

In [444]:
%%sql result_5a_min << 
SELECT min(salary)
FROM salaries;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_min


In [445]:
result_5a_min_query_str = "SELECT min(salary) FROM salaries;"
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                                QUERY PLAN                                                                 
-------------------------------------------------------------------------------------------------------------------------------------------
 Finalize Aggregate  (cost=10420.36..10420.37 rows=1 width=8) (actual time=47.397..113.609 rows=1 loops=1)
   ->  Gather  (cost=10420.15..10420.36 rows=2 width=8) (actual time=47.238..113.599 rows=3 loops=1)
         Workers Planned: 2
         Workers Launched: 2
         ->  Partial Aggregate  (cost=9420.15..9420.16 rows=1 width=8) (actual time=43.022..43.024 rows=1 loops=3)
               ->  Parallel Seq Scan on salaries  (cost=0.00..9080.12 rows=136012 width=8) (actual time=1.493..34.097 rows=108809 loops=3)
 Planning Time: 0.463 ms
 Execution Time: 113.686 ms
(8 rows)



In [446]:
result_5a_min_query_cost = 525.36
result_5a_min_query_timing = 4.493

In [447]:
grader.check("q5ai")

q5ai results: All test cases passed!

In [448]:
%%sql result_5a_avg <<
SELECT AVG(salary)
FROM salaries;  

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_avg


In [449]:
result_5a_avg_query_str = "SELECT AVG(salary) FROM salaries;  "
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                                QUERY PLAN                                                                 
-------------------------------------------------------------------------------------------------------------------------------------------
 Finalize Aggregate  (cost=10420.36..10420.37 rows=1 width=8) (actual time=79.432..85.591 rows=1 loops=1)
   ->  Gather  (cost=10420.15..10420.36 rows=2 width=32) (actual time=79.294..85.580 rows=3 loops=1)
         Workers Planned: 2
         Workers Launched: 2
         ->  Partial Aggregate  (cost=9420.15..9420.16 rows=1 width=32) (actual time=75.375..75.377 rows=1 loops=3)
               ->  Parallel Seq Scan on salaries  (cost=0.00..9080.12 rows=136012 width=8) (actual time=1.246..52.678 rows=108809 loops=3)
 Planning Time: 0.314 ms
 Execution Time: 85.699 ms
(8 rows)



In [450]:
result_5a_avg_query_cost = 525.36
result_5a_avg_query_timing = 5.450 

In [451]:
grader.check("q5aii")

q5aii results: All test cases passed!

### Question 5b:
Create an index on the `salary` column in the `salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [452]:
%%sql 
CREATE INDEX salary_inx
ON salaries(salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [453]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                                   QUERY PLAN                                                                    
-------------------------------------------------------------------------------------------------------------------------------------------------
 Result  (cost=0.48..0.49 rows=1 width=8) (actual time=0.098..0.099 rows=1 loops=1)
   InitPlan 1 (returns $0)
     ->  Limit  (cost=0.42..0.48 rows=1 width=8) (actual time=0.095..0.096 rows=1 loops=1)
           ->  Index Only Scan using salary_inx on salaries  (cost=0.42..19327.38 rows=326428 width=8) (actual time=0.094..0.094 rows=1 loops=1)
                 Index Cond: (salary IS NOT NULL)
                 Heap Fetches: 0
 Planning Time: 0.859 ms
 Execution Time: 0.158 ms
(8 rows)



In [454]:
result_5b_min_query_cost = 0.33
result_5b_min_query_timing =0.143

In [455]:
grader.check("q5bi")

q5bi results: All test cases passed!

In [456]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                                QUERY PLAN                                                                 
-------------------------------------------------------------------------------------------------------------------------------------------
 Finalize Aggregate  (cost=10420.36..10420.37 rows=1 width=8) (actual time=52.101..57.607 rows=1 loops=1)
   ->  Gather  (cost=10420.15..10420.36 rows=2 width=32) (actual time=51.965..57.597 rows=3 loops=1)
         Workers Planned: 2
         Workers Launched: 2
         ->  Partial Aggregate  (cost=9420.15..9420.16 rows=1 width=32) (actual time=47.967..47.968 rows=1 loops=3)
               ->  Parallel Seq Scan on salaries  (cost=0.00..9080.12 rows=136012 width=8) (actual time=1.177..35.678 rows=108809 loops=3)
 Planning Time: 0.467 ms
 Execution Time: 57.675 ms
(8 rows)



In [457]:
result_5b_avg_query_cost = 525.36
result_5b_avg_query_timing =  7.764 

In [458]:
grader.check("q5bii")

q5bii results: All test cases passed!

In the following questions, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `salary` column, drop the index before moving onto the next question.

In [459]:
%%sql 
DROP INDEX IF EXISTS salary_inx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 5c:
Given your findings from `Question 5a-b`, answer the following question. Assign the variable `q5c` to the correct answer choice.

1. Which of the following statements is true?
    1. An index on the column being aggregated in a query will always provide a performance enhancement.
    1. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
    1. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
    1. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

*Note:* Your answer should be formatted as follows: `q5c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q5c
points: 1
-->

In [460]:
q5c = ['D']

In [461]:
grader.check("q5c")

q5c results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 5d:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans.

<!--
BEGIN QUESTION
name: q5d
manual: true
points: 2
-->

I chose D as an answer because AVG and COUNT have to scan every single entry in the table and do additional math such as adding and diving which makes it to have high cost and time. On the other hand, MIN and MAX does not need additional calculation which makes it easier for the index to bring the first page of the index and the last page of the index. So this is why queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

<!-- END QUESTION -->



## Question 6: Clustered Indexes
In this question, we will inspect the impact that clustering our data on an index can have on a query's performance.

### Question 6a
Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [462]:
%%sql result_6a <<
SELECT playerid, yearid,teamid, ab
FROM batting
WHERE ab > 500;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8839 rows affected.
Returning data to local variable result_6a


In [463]:
# Do not delete/edit this cell
result_6a.DataFrame().sort_values(['playerid', 'yearid', 'teamid']).iloc[:1000].to_csv('results/result_6a.csv', index=False)

In [464]:
grader.check("q6ai")

q6ai results: All test cases passed!

In [465]:
result_6a_query_str = "SELECT playerid, yearid,teamid, ab FROM batting WHERE ab > 500;"
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                      QUERY PLAN                                                       
-----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on batting  (cost=106.69..1807.24 rows=9084 width=21) (actual time=0.725..6.132 rows=8839 loops=1)
   Recheck Cond: (ab > 500)
   Heap Blocks: exact=1311
   ->  Bitmap Index Scan on ab_idx  (cost=0.00..104.42 rows=9084 width=0) (actual time=0.555..0.556 rows=8839 loops=1)
         Index Cond: (ab > 500)
 Planning Time: 24.660 ms
 Execution Time: 6.587 ms
(7 rows)



In [466]:
result_6a_cost =2884.05
result_6a_timing =  13.708

In [467]:
grader.check("q6aii")

q6aii results: All test cases passed!

### Question 6b:
Cluster the `batting` table on its primary key (hint: use `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it. Re-inspect the query plan for the query from `Question 6a` and record the execution time and cost.

In [468]:
%sql \di

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
28 rows affected.


Schema,Name,Type,Owner
public,ab_idx,index,jovyan
public,allstarfull_pkey,index,jovyan
public,appearances_pkey,index,jovyan
public,awardsmanagers_pkey,index,jovyan
public,awardsplayers_pkey,index,jovyan
public,awardssharemanagers_pkey,index,jovyan
public,awardsshareplayers_pkey,index,jovyan
public,batting_pkey,index,jovyan
public,battingpost_pkey,index,jovyan
public,collegeplaying_pkey,index,jovyan


In [469]:
%%sql 
CLUSTER batting USING batting_pkey

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [470]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                      QUERY PLAN                                                       
-----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on batting  (cost=106.11..1792.72 rows=9009 width=21) (actual time=0.870..5.963 rows=8839 loops=1)
   Recheck Cond: (ab > 500)
   Heap Blocks: exact=1311
   ->  Bitmap Index Scan on ab_idx  (cost=0.00..103.86 rows=9009 width=0) (actual time=0.666..0.667 rows=8839 loops=1)
         Index Cond: (ab > 500)
 Planning Time: 0.475 ms
 Execution Time: 6.349 ms
(7 rows)



In [471]:
result_6b_cost = 2878.05
result_6b_timing = 14.464

In [472]:
grader.check("q6b")

q6b results: All test cases passed!

### Question 6c
Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table AND cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [473]:
%%sql 
CREATE INDEX ab_idx
ON batting(ab);

CLUSTER batting USING ab_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
(psycopg2.errors.DuplicateTable) relation "ab_idx" already exists

[SQL: CREATE INDEX ab_idx ON batting(ab);]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [474]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                      QUERY PLAN                                                       
-----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on batting  (cost=106.11..1792.72 rows=9009 width=21) (actual time=0.655..5.654 rows=8839 loops=1)
   Recheck Cond: (ab > 500)
   Heap Blocks: exact=1311
   ->  Bitmap Index Scan on ab_idx  (cost=0.00..103.86 rows=9009 width=0) (actual time=0.499..0.500 rows=8839 loops=1)
         Index Cond: (ab > 500)
 Planning Time: 0.374 ms
 Execution Time: 6.017 ms
(7 rows)



In [475]:
result_6c_cost = 1783.71
result_6c_timing = 2.394 

In [476]:
grader.check("q6c")

q6c results: All test cases passed!

### Question 6d
Given your findings from inspecting the query plans from `Question 6a-c`, answer the following question. Assign the variable `q6d` to a list of all statements that are true.

1. Consider the following statements:
    1. Clustering based on the `ab_idx` decreased the cost of the query.
    1. Clustering based on the `ab_idx` increased the cost of the query.
    1. Clustering based on the `ab_idx` increased the execution time of the query.
    1. Clustering based on the `ab_idx` decreased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the execution time of the query.
    1. None of the above
    
**Note:** Your answer should be formatted as follows: `q6d = ['A', 'B']`

<!--
BEGIN QUESTION
name: q6d
points: 2
-->

In [477]:
q6d = ['A','D']

In [478]:
grader.check("q6d")

q6d results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 6e:

Explain your answer to the previous part based on your knowledge from lectures, and your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q6e
manual: true
points: 2
-->

Custering based on ab_idx showed a decrese in execution time and cost because the query is filtering on a column  that is not a primary key and creating a unclustered index made the proceed to go faster. On the other hand, clustering based on batting_pkey did not change or even worsen the execution time due to the overhead of traversing the index since the query is filtering that is not a primary key.



<!-- END QUESTION -->



## Question 7: Cost of Index Management
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

### Question 7a:
Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [479]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
(psycopg2.errors.DuplicateColumn) column "added" of relation "salaries" already exists

[SQL: ALTER TABLE salaries ADD added boolean DEFAULT False;]
(Background on this error at: https://sqlalche.me/e/14/f405)


Next, run the provided update script and record the **wall time**.

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of this subpart.

In [492]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 3.85 ms, sys: 2.51 ms, total: 6.36 ms
Wall time: 7.03 s


[]

In [493]:
result_7a_timing = 6.66

In [494]:
grader.check("q7a")

q7a results: All test cases passed!

Before adding an index to the salaries table and re-timing the updates, delete all the rows that were added to the table from the update script.

In [495]:
%%sql
DELETE FROM salaries
WHERE added = 'true';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
300000 rows affected.


[]

### Question 7b:
Now, create an index on the `salary` column and record the **wall time** after executing the update script.

In [496]:
%%sql 
CREATE INDEX salary_idx
ON salaries(salary)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
(psycopg2.errors.DuplicateTable) relation "salary_idx" already exists

[SQL: CREATE INDEX salary_idx ON salaries(salary)]
(Background on this error at: https://sqlalche.me/e/14/f405)


**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of last subpart.

In [497]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 3.88 ms, sys: 1.48 ms, total: 5.36 ms
Wall time: 7.19 s


[]

In [498]:
result_7b_timing = 8.23

In [499]:
grader.check("q7b")

q7b results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 7c:
What difference did you notice when you added an index into the salaries table and re-timed the update? Why do you think it happened?

<!--
BEGIN QUESTION
name: q7c
manual: true
points: 2
-->

When it is insertion-heavy queries, having index will take longer wall time than without the index. This is because adding an index will store more data which makes it longer to be fully reflected on the table. Therefore, maintenance costs increases during updates which may lead to inaccurate query results.


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 8: Takeaway

In this project, we explored how the database system optimizes query execution and how users can futher tune the performance of their queries.

Familiarizing yourself with these optimization and tuning methods will make you a better data engineer. In this question, we'll ask you to recall and summarize these concepts. Who knows? Maybe one day it will help you during an interview or on a project.

In the following answer cell,
1. Name 3 methods you learned in this project. The method can be either the optimization done by the database system, or the fine tuning done by the user.
2. For each method, summarize how and why it can optimize query performance. Feel free to discuss any drawbacks, if applicable.

<!--
BEGIN QUESTION
name: q8
manual: true
points: 6
-->

1. Replace SUBQUERIES with JOIN : Although subqueries are useful, in general, a query that makes use of a subquery takes longer time to execute because we must first process that subquery and then process the outer query that uses the subquery.
2. Do CLUSTERING Based On Queries : Clustering on a primary key would not guarantee to enhance the performance if the query is conducting 'ordering'or 'filtering' that is not a primary key. Therefore, create a non clustered index and then activate clustering would develop the performance and cost.
3. Use INDEX Wisely : Indexes speed up the performance of a database but also takes space in memory. Especially, multicolumn indexes can store additional sorted pointers to other columns and can retrieve data faster from the table. Therefore, it is important to use index whenever it significantly improves the performance of your database. However, when there are frequent updates/deletes/insertions, using index would hinder performance because it takes longer time to update the database. 
4. USE FILTERS : Use filters such as WHERE or AND so that it reduces the amount of rows to store or scan. 


<!-- END QUESTION -->



## Congratulations! You have finished Project 2.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to register your written responses.

**For submission on Gradescope, you will need to submit BOTH the proj2.zip file genreated by the export cell and the results.zip file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj2.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [500]:
from IPython.display import display, FileLink

!zip -r results.zip results
results_file = FileLink('./results.zip', result_html_prefix="Click here to download: ")
display(results_file)

updating: results/ (stored 0%)
updating: results/result_1a.csv (deflated 53%)
updating: results/result_1c.csv (deflated 64%)
updating: results/result_1di_view.csv (deflated 37%)
updating: results/result_1di_no_view.csv (deflated 37%)
updating: results/result_1ei.csv (deflated 64%)
updating: results/result_1eii.csv (deflated 37%)
updating: results/result_4a.csv (deflated 59%)
updating: results/result_4ei.csv (deflated 62%)
updating: results/result_4eiii.csv (deflated 64%)
updating: results/result_6a.csv (deflated 72%)


/home/jovyan/fa22/proj/proj2/results.zip

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [501]:
grader.check_all()

q0 results: All test cases passed!

q1a results: All test cases passed!

q1bi results: All test cases passed!

q1bii results: All test cases passed!

q1c results: All test cases passed!

q1di results: All test cases passed!

q1dii results: All test cases passed!

q1ei results: All test cases passed!

q1eii results: All test cases passed!

q1eiii results: All test cases passed!

q1eiv results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q2c results: All test cases passed!

q3a results: All test cases passed!

q3b results: All test cases passed!

q3c results: All test cases passed!

q4ai results: All test cases passed!

q4aii results: All test cases passed!

q4b results: All test cases passed!

q4c results: All test cases passed!

q4di results: All test cases passed!

q4ei_part1 results: All test cases passed!

q4ei_part2 results: All test cases passed!

q4eii results: All test cases passed!

q4eiii_part1 results: All test cases passe

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [502]:
# Save your notebook first, then run this cell to export your submission.
grader.export()